# TensorFlow training on Azure GPU VM.
This notebook demonstrates how to use Azure Machine Learning service to orchestrate an end-to-end Machine Learning workflow from data preparation to model operationalization.


## Scenario

We will train a custom image classification model to automatically classify the type of land shown in aerial images of 224-meter x 224-meter plots. Land use classification models can be used to track urbanization, deforestation, loss of wetlands, and other major environmental trends using periodically collected aerial imagery. The images used in this lab are based on imagery from the U.S. National Land Cover Database. U.S. National Land Cover Database defines six primary classes of land use: *Developed*, *Barren*, *Forested*, *Grassland*, *Shrub*, *Cultivated*. Example images in each land use class are shown here:

Developed | Cultivated | Barren
--------- | ------ | ----------
![Developed](images/developed1.png) | ![Cultivated](images/cultivated1.png) | ![Barren](images/barren1.png)

Forested | Grassland | Shrub
---------| ----------| -----
![Forested](images/forest1.png) | ![Grassland](images/grassland1.png) | ![Shrub](images/shrub1.png)

We are going to employ a machine learning technique called transfer learning. Transfer learning is one of the fastest (code and run-time-wise) ways to start using deep learning. It allows to reuse knowledge gained while solving one problem to a different but related problem. For example, knowledge gained while learning to recognize cars could apply when trying to recognize trucks. Transfer Learning makes it feasible to train very effective ML models on relatively small training data sets, which is our case.

Although the primary goal of this lab is to understand how to use Azure ML to orchestrate TensorFlow training rather then to dive into Deep Learning techniques, ask the instructor if you want to better understand the approach utilized in the lab in more detail.

![Transfer Learing](images/TransferL.png)








## Lab flow

During the lab we will walk through the full end-to-end machine learning workflow.

![MLWorkflow](images/MLWorkflow.png)


- We will first develop data preparation and modeling routines in a local development environment, using a small development set of images to experiment and to understand the code. 

- After the code has been validated we will use Azure ML to deploy and run the data preparation and training scripts on Azure GPU cluster. We will run a number of concurrent training jobs to fine tune the model's hyper parameters

- Finally, we will operationalize the best performing version of the model.


We will use Azure Machine Learning service to orchestrate the above workflow.


## Prepare training and validation data

### Download the development dataset

In [ ]:
%%sh
wget -nv https://azureailabs.blob.core.windows.net/aerialtar/aerialtiny.tar.gz
tar -xzf aerialtiny.tar.gz


### Create bottleneck features

We will use the **ResNet50** DNN, pre-trained on the ImageNet dataset to extract features from input images. We will only instantiate the convolutional part of **ResNet50**, everything up to the fully-connected layers. We will then run this "stripped down" network (a.k.a featurizer) on training and validation images and store the output - the so called bottleneck features - in memory. Then we will train a small fully connected neural network on the bottleneck features. The output of the featurizer is a vector of 2048 numbers, resulting in a very small data set as compared to the original image dataset. As such, it is feasible to store it in memory. 

To feed data into the featurizer we will implement a custom generator class - `ImageGenerator`. It will yield batches of images - as Numpy arrays - preprocessed to the format required by `ResNet50` - Caffe style image encoding. Although, we could load all images from the small development set into memory, this approach would not scale to the larger data set we will be using later in the lab. As such we need a method of reading and pre-processing images in smaller batches.

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications import resnet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical

import numpy as np
import random

# This is a generator that yields batches of preprocessed images
class ImageGenerator(tf.keras.utils.Sequence):    
    
    def __init__(self, img_dir, preprocess_fn=None, batch_size=64):
        
        # Create the dictionary that maps class names into numeric labels 
        folders = os.listdir(img_dir)
        folders.sort()
        indexes = range(len(folders))
        label_map = {key: value for (key, value) in zip(folders, indexes)}
        self.num_classes = len(label_map)
        
        # Create a list of all images in a root folder with associated numeric labels
        labeled_image_list = [(os.path.join(img_dir, folder, image), label_map[folder]) 
                              for folder in folders 
                              for image in os.listdir(os.path.join(img_dir, folder))
                              ]
        # Shuffle the list
        random.shuffle(labeled_image_list)
        # Set image list and associated label list
        self.image_list, self.label_list = zip(*labeled_image_list) 
        # Set batch size
        self.batch_size = batch_size
       
        # Set the pre-processing function passed as a parameter
        self.preprocess_fn = preprocess_fn
        
        # Set number of batches
        self.n_batches = len(self.image_list) // self.batch_size
        if len(self.image_list) % self.batch_size > 0:
            self.n_batches += 1
            
    def __len__(self):
        
        return self.n_batches
    
    def __getitem__(self, index):
        pathnames = self.image_list[index*self.batch_size:(index+1)*self.batch_size]
        images = self.__load_images(pathnames)
        
        return images
    
    # Load a set of images passed as a parameter into a NumPy array
    def __load_images(self, pathnames):
        images = []
        for pathname in pathnames:
            img = image.load_img(pathname, target_size=(224,224,3))
            img = image.img_to_array(img)
            images.append(img)
        images = np.asarray(images)
        if self.preprocess_fn != None:
            images = self.preprocess_fn(images)   
        
        return images
    
    # Return labels in one-hot encoding
    def get_labels(self):
        
        return to_categorical(np.asarray(self.label_list), self.num_classes)

In [ ]:
# Create bottleneck featurs

train_images_dir = 'aerialtiny/train'
valid_images_dir = 'aerialtiny/valid'

train_generator = ImageGenerator(train_images_dir, resnet50.preprocess_input)
valid_generator = ImageGenerator(valid_images_dir, resnet50.preprocess_input)

featurizer = resnet50.ResNet50(
            weights = 'imagenet', 
            input_shape=(224,224,3), 
            include_top = False,
            pooling = 'avg')

train_features = featurizer.predict_generator(train_generator, verbose=1)
train_labels = train_generator.get_labels()

valid_features = featurizer.predict_generator(valid_generator, verbose=1)
valid_labels = valid_generator.get_labels()
 

## Train and evaluate

At this point, we have training and validation features and training and validation labels in in-memory Numpy arrays. 

In [ ]:
print(train_features.shape)
print(train_labels.shape)

We will now define, train and evaluate a small fully connected neural network on top of bottleneck features. We will encapsulate the network architecture in a utility function that takes a set of hyperparameters as input. This will make it easier to experiment with different hyperparameter combinations.

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten, Input
from tensorflow.keras.regularizers import l1_l2


def fcn_classifier(input_shape=(2048,), units=512, classes=6,  l1=0.01, l2=0.01):
    features = Input(shape=input_shape)
    x = Dense(units, activation='relu')(features)
    x = Dropout(0.5)(x)
    y = Dense(classes, activation='softmax', kernel_regularizer=l1_l2(l1=l1, l2=l2))(x)
    model = Model(inputs=features, outputs=y)
    model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
    return model
    

The next step is to instantiate the model ...

In [ ]:
model = fcn_classifier(input_shape=(2048,), units=1024, l1=0.006, l2=0.006)
model.summary()

... and start the training.

In [ ]:
model.fit(train_features, train_labels,
          batch_size=64,
          epochs=20,
          shuffle=True,
          validation_data=(valid_features, valid_labels))
          
          

Our code is now debugged and ready. We will now move to the next part of the lab in which we are going to use Azure Machine Learning service to train the model on a larger dataset using Azure GPU VM and then deploy the model to Azure Kubernetes Service.